# XGBoost-Feature Selection- Feature Engineering - Parameter Tuning

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import optuna

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time

path='/kaggle/input/tabular-playground-series-oct-2021/'
df_training = pd.read_csv(path+'train.csv')
#READING TEST DATASET AND SUBMISSION FILE
df_test=pd.read_csv(path+'test.csv')
df_submission=pd.read_csv(path+'sample_submission.csv')

In [ ]:
df_training.head()

In [ ]:
df_training.shape

# Memory Reduction

In [ ]:
for column in df_training.columns:
    if df_training[column].dtype == "float64":
        df_training[column]=pd.to_numeric(df_training[column], downcast="float")
    if df_training[column].dtype == "int64":
        df_training[column]=pd.to_numeric(df_training[column], downcast="integer")

In [ ]:
df_training.head()

In [ ]:
for column in df_test.columns:
    if df_test[column].dtype == "float64":
        df_test[column]=pd.to_numeric(df_test[column], downcast="float")
    if df_test[column].dtype == "int64":
        df_test[column]=pd.to_numeric(df_test[column], downcast="integer")

In [ ]:
df_test.head()

In [ ]:
X = df_training.drop('target', axis=1).copy()
X.head()

In [ ]:
y = df_training['target'].copy()
y.head()

In [ ]:
X_test = df_test.copy()
X_test.head()

In [ ]:
#new_f1= df_training.groupby(['f179'])['target'].mean()
#X["new_f1"] = X["f22"]*X["f53"]
#new_f2= df_training.groupby(['f69'])['target'].mean()
#new_f3= df_training.groupby(['f58'])['target'].mean()
#new_f4= df_training.groupby(['f136'])['target'].mean()
#new_f5= df_training.groupby(['f156'])['target'].mean()
#new_f6= df_training.groupby(['f78'])['target'].mean()
#new_f7= df_training.groupby(['f8'])['target'].mean()
#new_f8= df_training.groupby(['f214'])['target'].mean()
#feature_eng = [new_f1,new_f2,new_f3,new_f4,new_f5,new_f6,new_f7,new_f8]

#newdf = pd.DataFrame(feature_eng)
#newdf.head()

# Feature Selection


For details about feature selection, please see my notebooks:

Feature Selection using Random Forest: https://www.kaggle.com/adizafar/feature-importance-via-random-forest

Feature Selection using Genetic Algorithm:https://www.kaggle.com/adizafar/feature-selection-using-genetic-algorithm

In [ ]:
features = ["f179", "f69","f58", "f136", "f214","f156", "f78", "f8"]
useful_features = ["f22","f179","f69","f156","f58","f136","f214"]

In [ ]:
#useful_features = ["f22", "f179", "f69",
                   #"f58", "f214", "f78", "f136", "f156", "f8", "f3", "f77", "f200", "f92", "f185", "f142", "f115", "f284"]
#n_features=['f22', 'f179', 'f69', 'f58', 'f78', 'f44', 'f138', 'f139', 'f144' , 'f146', 'f157', 'f158', 'f160', 'f53', 'f170']
#useful_features_n = ['f22', 'f53', 'f170']
#n_features=['f22', 'f30', 'f53', 'f56', 'f126', 'f163']
X["mean"] = X[features].mean(axis=1)
X["std"] = X[features].std(axis=1)
X['variance']=X[features].var(axis=1)
X["min"] = X[features].min(axis=1)
X["max"] = X[features].max(axis=1)

X_test["mean"] = X_test[features].mean(axis=1)
X_test["std"] = X_test[features].std(axis=1)
X_test['variance']=X_test[features].var(axis=1)
X_test["min"] = X_test[features].min(axis=1)
X_test["max"] = X_test[features].max(axis=1)


#X_test["mean"] = X_test.mean(axis=1)
#X_test["std"] = X_test.std(axis=1)
#X_test['variance']=X_test.var(axis=1)

#X_test["mean"] = X_test[useful_features_n].mean(axis=1)
#X_test["std"] = X_test[useful_features].std(axis=1)
#X_test["min"] = X_test[useful_features].min(axis=1)
#X_test["max"] = X_test[useful_features].max(axis=1)
#X_test["new_f1"] = X_test["f22"]*X_test["f53"]
#X_test["new_f2"] = X_test["f22"]*X_test["f170"]
#X_test["new_f3"] = X_test["f170"]*X_test["f53"]


#X['std'] = X.std(axis=1)
#X['min'] = X.min(axis=1)
#X['max'] = X.max(axis=1)

#X_test['std'] = X_test.std(axis=1)
#X_test['min'] = X_test.min(axis=1)
#X_test['max'] = X_test.max(axis=1)



In [ ]:
X.shape

# Feature Engineering

For feature engineering, please see the reference notebook

I have utilized the work of: https://www.kaggle.com/motchan/tps-oct-2021-kmeans

In [ ]:
from sklearn.cluster import KMeans
feature_cols = [col for col in df_test.columns.tolist()]
n_clusters_1 = 6
cd_feature = True # cluster distance instead of cluster number
cluster_cols = [f"cluster{i+1}" for i in range(n_clusters_1)]
kmeans = KMeans(n_clusters=n_clusters_1, n_init=50, max_iter=500, random_state=42)

if cd_feature:
    # train
    X_cd = kmeans.fit_transform(df_training[useful_features])
    X_cd = pd.DataFrame(X_cd, columns=cluster_cols, index=df_training.index)
    df_training = df_training.join(X_cd)
    # test
    X_cd = kmeans.transform(df_test[useful_features])
    X_cd = pd.DataFrame(X_cd, columns=cluster_cols, index=df_test.index)
    df_test = df_test.join(X_cd)
    
else:
    # train
    df_training["cluster"] = kmeans.fit_predict(train[useful_features])
    # test
    df_test["cluster"] = kmeans.predict(test[useful_features])
    
    # one-hot encode
    ohe = OneHotEncoder()
    X_ohe = ohe.fit_transform(np.array(df_training["cluster"]).reshape(-1,1)).toarray()
    T_ohe = ohe.transform(np.array(df_test["cluster"]).reshape(-1,1)).toarray()

    X_ohe = pd.DataFrame(X_ohe, columns=cluster_cols, index=train.index)
    T_ohe = pd.DataFrame(T_ohe, columns=cluster_cols, index=test.index)

    df_training = pd.concat([train, X_ohe],axis=1)
    df_test = pd.concat([test, T_ohe],axis=1)

feature_cols += cluster_cols
df_training.head()

In [ ]:
X["cluster1"] = df_training['cluster1']
X["cluster2"] = df_training['cluster2']
X["cluster3"] = df_training['cluster3']
X["cluster4"] = df_training['cluster4']
X["cluster5"] = df_training['cluster5']
X["cluster6"] = df_training['cluster6']


In [ ]:
features_n = ["f22","f179", "f69","f58", "f136", "f214","f156", "f78", "f8","min",
              "max","mean","std","variance","cluster1","cluster2","cluster3","cluster4","cluster5","cluster6"]


In [ ]:
X["new_f1"] = (X["cluster1"])/(X["cluster2"])
X["new_f2"] = (X["cluster3"])/(X["cluster4"])
X["new_f3"] = (X["cluster5"])/(X["cluster6"])

In [ ]:
features_n_f = ["f22","f179", "f69","f58", "f136", "f214","f156", "f78", "f8","min",
              "max","mean","std","variance","cluster1","cluster2","cluster3","cluster4",
              "cluster5","cluster6","new_f1","new_f2","new_f3"]


In [ ]:
X[features_n_f].head()

# Parameter Tuning using Bayesian Optimization

For parameter tuning, please refer to the following notebook: 

https://www.kaggle.com/adizafar/xgboost-parameter-tuning-bayesian-optimization

In [ ]:
#{'colsample_bytree': 0.5805849806605896,
#'gamma': 2.467408277656332, 'max_depth': 8.0, 'min_child_weight': 7.0, 'reg_alpha': 156.0, 'reg_lambda': 0.2993899607673447}
params = {
   # 'max_depth': 6,
    'max_depth': 8,
    'n_estimators': 9500,
    'learning_rate': 0.007279718158350149,
    #'learning_rate': 0.05,
    'subsample': 0.7,
    #'colsample_bytree': 0.2,
    'colsample_bytree': 0.5805849806605896,
    'colsample_bylevel': 0.6000000000000001,
    #'min_child_weight': 56.41980735551558,
    'min_child_weight': 7.0,
    #'reg_lambda': 75.56651890088857,
    'reg_lambda': 0.2993899607673447,
    #'reg_alpha': 0.11766857055687065,
    'reg_alpha': 156.0,
    #'gamma': 0.6407823221122686
    'gamma': 2.467408277656332
    }

# Modeling

In [ ]:
%%time
kf = StratifiedKFold(n_splits=7, shuffle=True, random_state=0)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X[features_n_f], y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    model = XGBClassifier(**params,
                            booster= 'gbtree',
                            eval_metric = 'auc',
                            tree_method= 'gpu_hist',
                            predictor="gpu_predictor",
                            use_label_encoder=False)
    
    model.fit(X_train,y_train,
              eval_set=[(X_valid,y_valid)],
              early_stopping_rounds=100,
              verbose=False)
    
    pred_valid = model.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('||'*40)
    
    test_preds = model.predict_proba(X_test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

# Submission

In [ ]:
ss = dt.fread('../input/tabular-playground-series-oct-2021/sample_submission.csv').to_pandas()


In [ ]:
predictions = np.mean(np.column_stack(preds),axis=1)

ss['target'] = predictions
ss.to_csv('./first_sub_paratun_feat_selec.csv', index=False)
ss.head()